In [4]:
from Utils.my_utils import *
from Recommenders.GraphBased.P3alphaRecommender import *
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.NonPersonalizedRecommender import *
import os
from random import randint, seed

In [6]:
# setta anche il seed di random per sicurezza
seed = randint(1,1000)
print("Seed: " +str(seed))
np.random.seed(seed)
users, items, users_to_eval = readData()
URM = newPreProcess()
URM_train_full, URM_train, URM_test, URM_validation = splitURM(URM)

Seed: 251


In [7]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 3026 (23.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2494 (19.1%) Users that have less than 1 test interactions


In [8]:
output_folder_path = "result_experiments/"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

saved_folder_path = "result_best_models/"
if not os.path.exists(saved_folder_path):
    os.makedirs(saved_folder_path)
    
recommender_class = P3alphaRecommender    
n_cases = 10
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [9]:
runHyperparameterSearch_Collaborative(recommender_class,
       URM_train = URM_train,
       URM_train_last_test = URM_train_full,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

SearchBayesianSkopt: Resuming 'P3alphaRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 752, 'alpha': 0.7270844505807418, 'normalize_similarity': False}
P3alphaRecommender: URM Detected 874 ( 6.7%) users with no interactions.
P3alphaRecommender: URM Detected 446 ( 2.0%) items with no interactions.
P3alphaRecommender: Similarity column 22348 (100.0%), 1291.63 column/sec. Elapsed time 17.30 sec
EvaluatorHoldout: Processed 9999 (100.0%) in 8.46 sec. Users per second: 1182
SearchBayesianSkopt: New best config found. Config 0: {'topK': 752, 'alpha': 0.7270844505807418, 'normalize_similarity': False} - results: PRECISION: 0.0528753, PRECISION_RECALL_MIN_DEN: 0.1093307, RECALL: 0.0988938, MAP: 0.0220347, MAP_MIN_DEN: 0.0459663, MRR: 0.1551842, NDCG: 0.0883725, F1: 0.0689078, HIT_RATE: 0.3746375, ARHR_ALL_HITS: 0.1845444, NOVELTY: 0.0044749, AVERAGE_POPULARITY: 0.4459966, DIVERSITY_MEAN_INTER_L

In [10]:
# load the model with all best parameters
recommender_object = P3alphaRecommender(URM_train_full)
recommender_object.load_model(output_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_best_model_last.zip")

P3alphaRecommender: URM Detected 625 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 228 ( 1.0%) items with no interactions.
P3alphaRecommender: Loading model from file 'result_experiments/P3alphaRecommender_best_model_last.zip'
P3alphaRecommender: Loading complete


In [21]:
# recommend items to users
recommendations = recommender_object.recommend(users_to_eval, cutoff=10)

In [22]:
# evaluate recommender
evaluate_algorithm(URM_test, recommender_object)

MAP = 0.0788


In [23]:
newStoreRecommendations(users_to_eval, recommendations)

In [ ]:
# save the model with all best parameters overwriting the old one
# recommender_object.save_model(saved_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_my_own_save.zip")